In [41]:
import os
USER = !whoami
USER = USER[0]
OUTPUT_PATH_BASE = '/user/{USER}'.format(USER=USER)

### Local python version

In [4]:
import platform
platform.python_version()

'2.7.14'

### Let's check what's running on the clusters
Note that even though the shebang points to python 2.7, this is not enough to invoke the 2.7 environment

In [5]:
%%writefile one_line.txt
hello

Overwriting one_line.txt


In [116]:
%%writefile mrjob_versions_test.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from mrjob.compat import jobconf_from_env
from mrjob.job import MRJob
from mrjob.step import MRStep
import numpy as np
from collections import Counter
import platform
class MRCountLinesByFile(MRJob):

    def mapper(self, _, line):
        yield None, str(platform.python_version())
        yield None, str(np.log(2))


        
    def steps(self):
        JOBCONF_STEP = {        
           'mapreduce.job.reduces': '0' 
        }
        return [MRStep(
            jobconf=JOBCONF_STEP,    
            mapper = self.mapper
        )]

if __name__ == '__main__':
    MRCountLinesByFile.run()

Overwriting mrjob_versions_test.py


### Invoke python 2.7

Make a zip file of the virtual environemtn and ship it with your MRJob.   
You can conda install into your envirnment whatever packages you want. 

There is a PR to include numpy in the runner.sh script. 
### Or, skip the next block and just use the one in my hdfs dir which includes numpy.


In [82]:
%%bash

USERNAME=<your altiscale username>
ENV_NAME=py27

cd ~/.conda/envs/py27/
zip -r ~/py27.zip *
cd ~
hdfs dfs -mkdir /user/$USERNAME/virtualenv/
hdfs dfs -put py27.zip /user/$USERNAME/virtualenv/

In [119]:
pyArchive = 'hdfs:///user/kylehamilton/virtualenv/py27.zip#py27'
pyBin = './py27/bin/python'

In [121]:
OUTPUT_PATH_1 = os.path.join(OUTPUT_PATH_BASE,'tests1')
!hadoop fs -rm -r {OUTPUT_PATH_1}
!python mrjob_versions_test.py \
        -r hadoop one_line.txt \
        --python-archive={pyArchive} \
        --python-bin={pyBin} \
        --output-dir={OUTPUT_PATH_1} \
        --no-output
        

rm: `/user/kylehamilton/tests1': No such file or directory
No configs found; falling back on auto-configuration
Creating temp directory /tmp/mrjob_versions_test.kylehamilton.20171011.230304.969426
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Copying local files to hdfs:///user/kylehamilton/tmp/mrjob/mrjob_versions_test.kylehamilton.20171011.230304.969426/files/...
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Running step 1 of 1...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob4578348651553513670.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/timeline/
  Connecting to ResourceManager at rm-ia.s3s.altiscale.com/10.251.255.108:8032
  Connecting to Application History server at rm-ia.s3s.altiscale.com/10.251.255.108:10200
  Tim

In [122]:
!hadoop fs -cat {OUTPUT_PATH_1}/*

null	"2.7.14"
null	"0.69314718056"


# Hadoop streaming example

In [59]:
%%writefile test_py_27.py
#!/usr/bin/env /opt/rh/python27/root/usr/bin/python2.7

import os
import time
import subprocess
import platform
import numpy as np

if __name__ == "__main__":
    
    print(str(platform.python_version()))
    print(np.log(2))
    python_default_runtime_path = subprocess.Popen(["which", "python"], stdout=subprocess.PIPE)
    result_lines = python_default_runtime_path.communicate()[0]
    lines = result_lines.decode("utf-8")
    print(lines)

    list_runtime_files_result = subprocess.Popen(["ls", "-alR"], stdout=subprocess.PIPE)
    result_lines = list_runtime_files_result.communicate()[0]
    lines = result_lines.decode("utf-8")
    print(lines)

    env_proc_result = subprocess.Popen(["env"], stdout=subprocess.PIPE)
    result_lines = env_proc_result.communicate()[0]
    lines = result_lines.decode("utf-8")
    print(lines)

    mount = subprocess.check_output(["mount" ,"-v"])
    lines = mount.decode("utf-8").splitlines()
    mount_lists = map(lambda line: line.split()[2], lines)
    for fpath in mount_lists:
        print(fpath)

    print(os.environ)

Overwriting test_py_27.py


In [60]:
%%bash
#!/bin/bash

HADOOP_VERSION=2.7.1
TEST_PID=$$
HDFS_TEST_OUTPUT=/tmp/scl-kyle-test_${TEST_PID}

# Generate a random sample and upload to HDFS
# Default goes to /user/$USER/randomsample.txt
echo -e "1\n2\n3\n4\n5\n6\n7\n8\n9" > "randomsample.txt"

# Ignore HDFS errors here
hdfs dfs -put randomsample.txt
# Ignore error from HDFS destination existance, can ignore
hdfs dfs -rm -r $HDFS_TEST_OUTPUT

# Launch Hadoop Streaming with SCL configured properly
# Reducer # is set to 0 on purpose to avoid shuffle + sort on results
hadoop jar /opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar -files test_py_27.py -mapper test_py_27.py -numReduceTasks 0 -input /user/$USER/randomsample.txt -output $HDFS_TEST_OUTPUT -cmdenv LD_LIBRARY_PATH=/opt/rh/python27/root/usr/lib64

packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob7401991266607017549.jar tmpDir=null


put: `randomsample.txt': File exists
rm: `/tmp/scl-kyle-test_53888': No such file or directory
17/10/11 20:43:45 INFO impl.TimelineClientImpl: Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/timeline/
17/10/11 20:43:45 INFO client.RMProxy: Connecting to ResourceManager at rm-ia.s3s.altiscale.com/10.251.255.108:8032
17/10/11 20:43:45 INFO client.AHSProxy: Connecting to Application History server at rm-ia.s3s.altiscale.com/10.251.255.108:10200
17/10/11 20:43:46 INFO impl.TimelineClientImpl: Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/timeline/
17/10/11 20:43:46 INFO client.RMProxy: Connecting to ResourceManager at rm-ia.s3s.altiscale.com/10.251.255.108:8032
17/10/11 20:43:46 INFO client.AHSProxy: Connecting to Application History server at rm-ia.s3s.altiscale.com/10.251.255.108:10200
17/10/11 20:43:50 INFO lzo.GPLNativeCodeLoader: Loaded native gpl library from the embedded binaries
17/10/11 20:43:50 INFO lzo.LzoCodec: Successfully loaded & ini

In [61]:
!hdfs dfs -cat /tmp/scl-kyle-test_53888/part-00000

2.7.8	
0.69314718056	
/usr/bin/python	
	
.:	
total 164	
drwxr-s--- 3 kylehamilton yarn   4096 Oct 11 20:44 .	
drwxr-s--- 6 kylehamilton yarn   4096 Oct 11 20:44 ..	
-rw-r----- 1 kylehamilton yarn   1012 Oct 11 20:44 .job.xml.crc	
-rw------- 1 kylehamilton yarn    129 Oct 11 20:44 container_tokens	
lrwxrwxrwx 1 kylehamilton yarn    113 Oct 11 20:44 job.jar -> /var/storage/nm-sdg1/nm-local/usercache/kylehamilton/appcache/application_1506640654827_2674/filecache/11/job.jar	
-rw-r----- 1 kylehamilton yarn 128505 Oct 11 20:44 job.xml	
-rwx------ 1 kylehamilton yarn   7095 Oct 11 20:44 launch_container.sh	
lrwxrwxrwx 1 kylehamilton yarn     79 Oct 11 20:44 test_py_27.py -> /var/storage/nm-sde1/nm-local/usercache/kylehamilton/filecache/20/test_py_27.py	
drwxr-s--- 2 kylehamilton yarn   4096 Oct 11 20:44 tmp	
	
./tmp:	
total 80	
drwxr-s--- 2 kylehamilton yarn  4096 Oct 11 20:44 .	
drwxr-s--- 3 kylehamilton yarn  4096 Oct 11 20:44 ..	
-rwxr-x--x 1 kylehamilton yarn 69643 Oct 11 20:44 unpacked-6